In [286]:
import cv2
import numpy as np

In [287]:
cap = cv2.VideoCapture('Good_horizon_2.mp4')

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Frames per second: {fps}")
frame_delay = int(1000 / fps)  # 毫秒

Frames per second: 30.0


In [288]:
kernel = np.ones((3, 3), np.uint8)

def Denoise(RGBImage, ksize):
    img_blur= cv2.medianBlur(RGBImage, ksize)
    return img_blur

def Open(GrayImage, erode, dilate):
    img_erode = cv2.erode(GrayImage, kernel, iterations = erode)
    img_dilate = cv2.dilate(img_erode, kernel, iterations = dilate)
    return img_dilate

def Close(GrayImage, erode, dilate):
    img_dilate = cv2.dilate(GrayImage, kernel, iterations = dilate)
    img_erode = cv2.erode(img_dilate, kernel, iterations = erode)
    return img_erode

In [289]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        print("Video is end.")
        break
    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    
    frame_denoise = Denoise(frame, 5)
    gray = cv2.cvtColor(frame_denoise, cv2.COLOR_RGB2GRAY)
    gray_HistEq = clahe.apply(gray)
    
    __, binary = cv2.threshold(gray_HistEq, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary_close = Open(binary, 5, 3)
    contours, __ = cv2.findContours(binary_close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if(area > 150000 and area < 200000):
            cv2.drawContours(frame, cnt, -1, (0, 255, 0), 4)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(frame_delay)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()